<a href="https://colab.research.google.com/github/mirjampaales/cool-ml-project/blob/main/named_entity_recognition/train_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Downloading data 
This can be skipped if using the notebook withing the .git repository as the repositories below are included there as submodules:

In [1]:
! git clone https://github.com/mukhal/xlm-roberta-ner.git 
! git clone https://github.com/TurkuNLP/turku-ner-corpus
! git clone https://github.com/ksirts/EstNER

Cloning into 'xlm-roberta-ner'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 312 (delta 165), reused 245 (delta 118), pack-reused 0
Receiving objects: 100% (312/312), 2.89 MiB | 10.43 MiB/s, done.
Resolving deltas: 100% (165/165), done.
Cloning into 'turku-ner-corpus'...
remote: Enumerating objects: 1611, done.
remote: Counting objects: 100% (1611/1611), done.
remote: Compressing objects: 100% (1515/1515), done.
remote: Total 1611 (delta 67), reused 1574 (delta 46), pack-reused 0
Receiving objects: 100% (1611/1611), 6.77 MiB | 13.13 MiB/s, done.
Resolving deltas: 100% (67/67), done.
Cloning into 'EstNER'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 2), reused 4 (delta 1), pack-reused 0
Unpacking objects: 100% (8/8), done.


## Data preparation

Data preparation to uniform formats across languages. As we will use the XLM-R finetuning code with the English dataset included, we don't have to worry about the format of English data.

The expected dataset format is a space-separated file where only the first and last column are looked at (word and its label), other columns are ignored. Sentences are separated by an empty line.

Finnish is easy, as it is a .tsv file with those two columns. Estonian is a completely different hierarchical JSON format and needs most preparation.


Additionally, we'll limit the named entity labels to just persons (PER), organizations (ORG) and locations (LOC), as those are common in all datasets.

In [31]:
import pandas as pd
import json

In [54]:
! mkdir et-data en-data fi-data merged-data

In [45]:
allowed_labels = ['O', 'B-LOC', 'I-LOC', 'B-ORG','I-ORG','B-PER','I-PER']

Estonian:

In [55]:
for split in ["dev","test","train"]:
  with open(f"EstNER/EstNER_v1_{split}.json", 'r') as f_in:
    data = json.loads(f_in.read())
  
  split = 'valid' if split=='dev' else split

  with open(f"et-data/{split}.txt", 'w') as f_out:
    for document in data:
      for sentence in document:
        for token in sentence:
          # Estonian has multi-part names sometimes marked as one token (e.g. New York). Those must be split to multiple rows to conform with the file format.
          words = token['word'].split()
          label = token['ner_1']

          label = label if label in allowed_labels else 'O'

          f_out.write(f"{words.pop(0)} {label}\n")

          if words: # name was multipart
            # if the first word is a named entity start (label B-*), others must be continuations
            if label.split('-')[0]=='B':
              label=f"I-{label.split('-')[1]}"
            
            while words:
              f_out.write(f"{words.pop(0)} {label}\n")
        f_out.write('\n')

Finnish:

In [58]:
for split in  ["dev","test","train"]:
  with open(f"turku-ner-corpus/data/conll/{split}.tsv", 'r') as f_in:
    data = f_in.readlines()
  
  split = 'valid' if split=='dev' else split

  with open(f"fi-data/{split}.txt", 'w') as f_out:
    for line in data:
      columns = line.strip().split('\t')
      if len(columns) >= 2:
        if columns[-1] in allowed_labels:
          f_out.write(f"{columns[0]} {columns[-1]}\n")
        else:
          f_out.write(f"{columns[0]} O\n")
      else:
        f_out.write(f"\n")
    f_out.write('\n')

English:

In [60]:
for split in  ["valid","test","train"]:
  with open(f"xlm-roberta-ner/data/coNLL-2003/{split}.txt", 'r') as f_in:
    data = f_in.readlines()

  with open(f"en-data/{split}.txt", 'w') as f_out:
    for line in data:
      columns = line.strip().split()
      if len(columns) >= 2:
        if columns[-1] in allowed_labels:
          f_out.write(f"{columns[0]} {columns[-1]}\n")
        else:
          f_out.write(f"{columns[0]} O\n")
      else:
        f_out.write(f"\n")
    f_out.write('\n')

Create merged datasets:

In [62]:
! for f in {valid,test,train}; do cat {et,en,fi}-data/$f.txt > merged-data/$f.txt; done

Distribution of tags across languages. We'll keep only PER, ORG, and LOC - these are supported by all languages.

## Finetuning XLM-R

In [63]:
! pip install -r xlm-roberta-ner/requirements.txt
! pip install wandb

     |████████████████████████████████| 306 kB 4.2 MB/s 
     |████████████████████████████████| 176 kB 52.1 MB/s 
     |████████████████████████████████| 90 kB 7.1 MB/s 
     |████████████████████████████████| 1.2 MB 54.7 MB/s 
     |████████████████████████████████| 895 kB 35.7 MB/s 
     |████████████████████████████████| 131 kB 72.9 MB/s 
     |████████████████████████████████| 79 kB 6.5 MB/s 
     |████████████████████████████████| 8.4 MB 28.9 MB/s 
     |████████████████████████████████| 138 kB 51.2 MB/s 
     |████████████████████████████████| 127 kB 51.1 MB/s 
  Created wheel for fairseq: filename=fairseq-0.9.0-cp37-cp37m-linux_x86_64.whl size=2168263 sha256=b11f82f8769e2f488162f7fe252b31434db1cf5726bfc4d29602d7ae48ab00ad
  Stored in directory: /root/.cache/pip/wheels/6b/27/e2/c55614da7eb71041bb08f02e8a302b869e51185eb7c575a604
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7434 sha256=27a609c4aed5737504d1acc7457465165b9fc9633d151757b0088b3ce48657c5
 

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 97 kB 5.2 MB/s 
     |████████████████████████████████| 140 kB 41.6 MB/s 
     |████████████████████████████████| 180 kB 49.4 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=59e550096cf29e0381001c1cbe804e66b058d918bbf2d052c3de5097aa754801
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=f3839d1c1d98183b3a997913bab201d6181db4a0bfea5e2eb107305efd2162c9
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


Checking which GPU resources we have:

In [ ]:
! nvidia-smi

Sat Dec 11 13:37:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Downloading the pretrained XLM-R model

In [ ]:
! mkdir model_dir
! mkdir pretrained_models 
! wget -P pretrained_models https://dl.fbaipublicfiles.com/fairseq/models/xlmr.base.tar.gz
! tar xzvf pretrained_models/xlmr.base.tar.gz  --directory pretrained_models/
! rm -r pretrained_models/xlmr.base.tar.gz

mkdir: cannot create directory ‘pretrained_models’: File exists
--2021-12-11 13:36:10--  https://dl.fbaipublicfiles.com/fairseq/models/xlmr.base.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512274718 (489M) [application/gzip]
Saving to: ‘pretrained_models/xlmr.base.tar.gz’

xlmr.base.tar.gz    100%[===================>] 488.54M  32.0MB/s    in 16s     

2021-12-11 13:36:26 (31.2 MB/s) - ‘pretrained_models/xlmr.base.tar.gz’ saved [512274718/512274718]

xlmr.base/
xlmr.base/dict.txt
xlmr.base/sentencepiece.bpe.model
xlmr.base/model.pt


Setting up Weights & Biases monitoring to keep an eye on GPU performance metrics (utilization, memory consumption, etc.)

In [ ]:
wandb.init()

wandb: Currently logged in as: liisaratsep (use `wandb login --relogin` to force relogin)


Finetuning with the same parameters as close to the original [XLM-R paper](https://arxiv.org/pdf/1911.02116.pdf) as possible. This finetunes the multilingual model, by changin the `data_dir` path, finetuning on a single language is possible.

PS: actual finetuning was done on UT HPC Rocket cluster, as it was faster, but the example below is completely functional.

In [ ]:
! python xlm-roberta-ner/main.py \
    --data_dir=./merged-data/  \
    --task_name=ner   \
    --output_dir=merged-model/   \
    --max_seq_length=128   \
    --num_train_epochs 10  \
    --do_eval \
    --warmup_proportion=0.0 \
    --pretrained_path pretrained_models/xlmr.base/ \
    --learning_rate 6e-5 \
    --do_train \
    --eval_on dev \
    --train_batch_size 32 \
    --dropout 0.2 \
    --train_batch_size 32

loading archive file pretrained_models/xlmr.base/
| dictionary: 250001 types
12/11/2021 14:10:54 - INFO - root -   *** Example ***
12/11/2021 14:10:54 - INFO - root -   guid: train-0
12/11/2021 14:10:54 - INFO - root -   tokens: 0 32232 107272 433 11532 22869 24317 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
12/11/2021 14:10:54 - INFO - root -   input_ids: 0 32232 107272 433 11532 22869 24317 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
12/11/2021 14:10:54 - INFO - root -   input_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 